### Context
### ...

### On Importe les librairies nécessaire 

In [1]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

### On Importe les données 

In [35]:
DataPlat = pd.read_csv('Data/TFT_Platinum_MatchData.csv');
DataPlat.combination[7]

"{'Blaster': 1, 'Cybernetic': 1, 'Infiltrator': 1, 'Set3_Blademaster': 1, 'Set3_Celestial': 1, 'Set3_Sorcerer': 1, 'StarGuardian': 1, 'Valkyrie': 1}"

In [11]:
DataPlat

,gameId,gameDuration,level,lastRound,Ranked,ingameDuration,combination,champion
0,KR_4291707834,1963.905273,6,27,5,1390.165771,"{'Cybernetic': 1, 'Demolitionist': 1, 'Infiltr...","{'Ziggs': {'items': [7], 'star': 1}, 'Ashe': {..."
1,KR_4291707834,1963.905273,8,37,3,1891.282715,"{'Blaster': 1, 'Chrono': 1, 'Cybernetic': 4, '...","{'Ziggs': {'items': [24], 'star': 3}, 'Fiora':..."
2,KR_4291707834,1963.905273,6,25,7,1279.461060,"{'Blaster': 1, 'Cybernetic': 1, 'DarkStar': 2,...","{'Fiora': {'items': [1], 'star': 1}, 'Shaco': ..."
3,KR_4291707834,1963.905273,7,38,2,1955.608521,"{'DarkStar': 1, 'Protector': 2, 'Set3_Blademas...","{'Poppy': {'items': [], 'star': 2}, 'Xayah': {..."
4,KR_4291707834,1963.905273,8,38,1,1955.608521,"{'Blaster': 1, 'Chrono': 5, 'DarkStar': 3, 'Pr...","{'TwistedFate': {'items': [36, 27], 'star': 3}..."
...,...,...,...,...,...,...,...,...
79995,KR_4296831363,2049.954102,7,34,4,1862.439209,"{'Chrono': 2, 'Cybernetic': 1, 'Protector': 2,...","{'TwistedFate': {'items': [58], 'star': 2}, 'X..."
79996,KR_4296831363,2049.954102,8,33,5,1799.791870,"{'Chrono': 2, 'Cybernetic': 1, 'Infiltrator': ...","{'Xayah': {'items': [5], 'star': 2}, 'Yasuo': ..."
79997,KR_4296831363,2049.954102,8,38,2,2041.694946,"{'Blaster': 1, 'DarkStar': 1, 'MechPilot': 1, ...","{'Poppy': {'items': [], 'star': 2}, 'Zoe': {'i..."
79998,KR_4296831363,2049.954102,8,35,3,1911.133057,"{'Blaster': 2, 'Chrono': 1, 'Demolitionist': 1...","{'Ziggs': {'items': [44, 48], 'star': 3}, 'Mal..."


In [3]:
DataChall = pd.read_csv('Data/TFT_Challenger_MatchData.csv');
DataChall

,gameId,gameDuration,level,lastRound,Ranked,ingameDuration,combination,champion
0,KR_4247538593,2142.470703,8,35,1,2134.272217,"{'DarkStar': 2, 'Protector': 4, 'Rebel': 1, 'S...","{'JarvanIV': {'items': [27], 'star': 3}, 'Sona..."
1,KR_4247538593,2142.470703,9,35,2,2134.272217,"{'Blaster': 2, 'Mercenary': 1, 'Rebel': 6, 'Se...","{'Malphite': {'items': [7], 'star': 2}, 'Yasuo..."
2,KR_4247538593,2142.470703,8,34,3,2073.459229,"{'Cybernetic': 1, 'DarkStar': 3, 'Demolitionis...","{'KaiSa': {'items': [99, 2, 23], 'star': 2}, '..."
3,KR_4247538593,2142.470703,8,33,4,1998.146729,"{'Blaster': 1, 'Cybernetic': 1, 'DarkStar': 1,...","{'KaiSa': {'items': [44, 37], 'star': 2}, 'Ann..."
4,KR_4247538593,2142.470703,9,33,5,1986.443237,"{'Blaster': 2, 'Demolitionist': 2, 'Mercenary'...","{'Ziggs': {'items': [], 'star': 1}, 'Yasuo': {..."
...,...,...,...,...,...,...,...,...
79994,KR_4357265434,2094.518555,7,33,4,1838.332764,"{'DarkStar': 2, 'Demolitionist': 2, 'Infiltrat...","{'KhaZix': {'items': [67, 26, 56], 'star': 2},..."
79995,KR_4357265434,2094.518555,8,33,5,1837.548096,"{'Blaster': 1, 'Chrono': 2, 'Cybernetic': 6, '...","{'Fiora': {'items': [35], 'star': 2}, 'Leona':..."
79996,KR_4357265434,2094.518555,9,33,6,1833.472046,"{'Chrono': 2, 'Cybernetic': 1, 'Demolitionist'...","{'TwistedFate': {'items': [], 'star': 2}, 'Mal..."
79997,KR_4357265434,2094.518555,8,31,7,1730.829712,"{'Blaster': 4, 'Chrono': 2, 'Cybernetic': 1, '...","{'Malphite': {'items': [], 'star': 1}, 'Graves..."
